# Bathroom Simulation

## Explanation interactive notebook
### This file is a jupyter notebook. Use it by installing python, and then follow the steps to set up jupyter notebook at [jupyter´s website](https://jupyter.org/install)

Here is where all explanations for everything in each file will be

## en_str/mod.rs
In this file I set out the structs and enums I will be using in main.rs

### What is a struct?
A struct is a collection of data that can be retrieved and changed. Structs can also implement functions however in this program they do not. For example:  
You could have a struct for a dog with the properties of name, age, and owner. In rust that would look like this: